In [1]:
import json
import os
import re
from pathlib import Path
from typing import Any, Dict, List, Union  # From mypy library

import numpy as np
import pandas as pd
import requests
import xmltodict  # type: ignore
from dotenv import find_dotenv, load_dotenv
from jinja2 import Template
from PIL import Image, ImageSequence  # type: ignore

from pandas.api.types import is_datetime64_any_dtype, is_numeric_dtype, is_integer_dtype, is_object_dtype, is_string_dtype

load_dotenv(find_dotenv())

api_key = os.getenv('XTRACTA_API_KEY')
database_id = os.getenv('XTRACTA_DATABASE_ID')
header_workflow = os.getenv('XTRACTA_HEADER_ID')
line_workflow = os.getenv('XTRACTA_LINE_ID')

# https://www.pythonlikeyoumeanit.com/Module5_OddsAndEnds/Modules_and_Packages.html

## Set up test environment

In [38]:
number_of_rows = 100

def delete_files(folder):
    file_generator = folder.glob('**/*')
    file_list = list(file_generator)
    for file in file_list:
        file.unlink()
    return list(file_list)

def move_files(samples_folder, file_to_copy, destination_folder):
    (destination_folder / file_to_copy).write_bytes((samples_folder / file_to_copy).read_bytes())
    return file_to_copy

def convert_to_numeric_and_date(df, dayfirst=True):
    for column in df.columns:
        if is_object_dtype(df[column]) or is_string_dtype(df[column]):
            try:
                df[column] = pd.to_numeric(df[column], downcast='integer')
            except:
                try:
                    df[column] = df[column].str.replace('$', '')
                    df[column] = df[column].str.replace(',', '')
                    df[column] = pd.to_numeric(df[column])
                except:
                    try:
                        df[column] = pd.to_datetime(df[column], dayfirst=dayfirst)
                    except:
                        pass
    return df


def random_dates(start, end, seed=1, replace=True, number_of_rows=100):
    dates = pd.date_range(start, end).to_series()
    return dates.sample(number_of_rows, replace=replace, random_state=seed).index
    
    
def dataframe_obfuscator(df, number_of_rows=100):
    for column in df.columns:
        if is_datetime64_any_dtype(df[column]):
            df[column] = random_dates(min(df[column]),max(df[column]), seed=1)
        elif is_integer_dtype(df[column]):
            df[column] = df[column].fillna(0)
            if min(df[column]) < max(df[column]):
                df[column] = np.random.randint(min(df[column]),max(df[column]),size=(number_of_rows))
            else:
                df[column] = min(df[column])
        elif is_numeric_dtype(df[column]):
            df[column] = df[column].fillna(0)
            df[column] = np.random.uniform(min(df[column]),max(df[column]),size=(number_of_rows))
        else:
            df[column] = 'random text'
    return df


def obfuscate_csv(data_file, dayfirst=True, number_of_rows=100):
    df = pd.read_csv(data_file, nrows=number_of_rows)
    df = convert_to_numeric_and_date(df)
    df = dataframe_obfuscator(df)
    df.to_csv(data_file, header=True, index=False)
    return df


def obfuscate_excel(data_file, dayfirst=True, number_of_rows=100):
    df = pd.read_excel(data_file, nrows=number_of_rows)
    display(df.head())
    df = convert_to_numeric_and_date(df)
    df = dataframe_obfuscator(df)
    df.to_excel(data_file, header=True, index=False)
    return df

In [81]:
p = Path.cwd()
dp = p / 'data'
sp = p / 'test_samples'
ip = p / 'input'
op = p / 'output'
jp = p / 'junk'


test_file = '20190131_invoice.pdf'

delete_files(ip)
delete_files(op)
move_files(sp, test_file, ip)


'20190131_invoice.pdf'

In [39]:
df = obfuscate_excel(dp / 'Purchase Order Master Data' / '20181105SP004 – PO Costing Report.xls')
df.head(5)

,District Code,Purchase Order Number,Purchase Order Item Number,Warehouse ID,Item Description Detail,Date of Purchase Order,Current Due Date,Original QTY UOI,Current QTY UOI,Original Net Price UOI,...,Purchase Officer Name,Requested By ID,Requested By Name,Authorised By ID,Authorised By Name,Request by Pos,New Requested,New Requested By,Unnamed: 35,Waiver Indicator
0,random text,random text,1,random text,random text,2017-03-19,2017-09-27,0,0,1.343214e+06,...,random text,83933,random text,86309,random text,331,13189.479587,random text,0,random text
1,random text,random text,1,random text,random text,2003-09-27,2004-04-06,0,0,7.908380e+06,...,random text,74018,random text,82311,random text,2522,68210.721985,random text,0,random text
2,random text,random text,1,random text,random text,2013-12-28,2014-07-08,0,0,7.529571e+06,...,random text,86084,random text,79174,random text,3895,55760.926420,random text,0,random text
3,random text,random text,1,random text,random text,2017-04-23,2017-11-01,0,0,3.649093e+06,...,random text,75600,random text,79990,random text,4152,81145.751779,random text,0,random text
4,random text,random text,1,random text,random text,2005-07-28,2006-02-05,0,0,6.816010e+06,...,random text,80777,random text,67247,random text,3866,51409.862595,random text,0,random text


,District Code,Purchase Order Number,Purchase Order Item Number,Warehouse ID,Item Description Detail,Date of Purchase Order,Current Due Date,Original QTY UOI,Current QTY UOI,Original Net Price UOI,...,Purchase Officer Name,Requested By ID,Requested By Name,Authorised By ID,Authorised By Name,Request by Pos,New Requested,New Requested By,Unnamed: 35,Waiver Indicator
0,random text,random text,1,random text,random text,2017-04-03,2017-10-12,0,0,5.768938e+06,...,random text,63757,random text,72194,random text,3392,22411.199978,random text,0,random text
1,random text,random text,1,random text,random text,2003-10-12,2004-04-21,0,0,5.857366e+06,...,random text,75428,random text,65981,random text,1000,74548.053953,random text,0,random text
2,random text,random text,1,random text,random text,2014-01-12,2014-07-23,0,0,2.919417e+06,...,random text,78695,random text,66043,random text,4594,18711.298160,random text,0,random text
3,random text,random text,1,random text,random text,2017-05-08,2017-11-16,0,0,5.391292e+06,...,random text,79285,random text,66695,random text,1115,47820.255969,random text,0,random text
4,random text,random text,1,random text,random text,2005-08-12,2006-02-20,0,0,8.018085e+06,...,random text,72091,random text,66498,random text,1080,85421.399989,random text,0,random text


In [40]:
df = obfuscate_csv(dp / 'Stock PO Master Data' / 'SP005.csv')
df.head()

,Purchase_Order_Number_Combined,District_Code,Warehouse_Identification,FPA_Group,FPA_Agreement_Number,FPA_Item_Number,Supplier_Price_Code1,UOI_Original_Quantity,UOI_Original_Net_Price1,UOI_Current_Quantity1,...,Total,Stock_Code,Item_Name_line,Stock_Description,Employee_Identification,FullName,Purchase_Order_Date,Supplier_Number,Supplier_Name,Item_Status_Code
0,random text,random text,random text,random text,0.162259,92,random text,107.328045,9.458306e+04,270.202673,...,3836.950016,8.422157e+05,random text,random text,random text,random text,2014-04-29,random text,random text,2
1,random text,random text,random text,random text,0.288029,250,random text,117.007044,7.874579e+06,399.332839,...,1888.459122,2.973084e+06,random text,random text,random text,random text,2000-11-06,random text,random text,3
2,random text,random text,random text,random text,0.667106,243,random text,387.005313,1.912920e+06,468.113125,...,2368.699692,3.876505e+05,random text,random text,random text,random text,2011-02-07,random text,random text,7
3,random text,random text,random text,random text,0.698380,240,random text,289.651560,4.896474e+06,38.559449,...,1031.245496,1.963139e+06,random text,random text,random text,random text,2014-06-03,random text,random text,5
4,random text,random text,random text,random text,0.804263,334,random text,426.569913,7.681263e+06,658.607700,...,5967.914098,1.493790e+06,random text,random text,random text,random text,2002-09-07,random text,random text,6


## Functions for interacting with Xtracta's API

### Upload file

Uploads a PDf or image file for extraction. The classifier field is used if you want to assign a specific classifier to the document rather than letting Xtracta make its own classification decision

In [83]:
def upload_file(api_key, workflow_id, filename, classifier=""):
    classifier_xml = (
        f'<field_data><field name="Classifier">{classifier}</field></field_data>'
    )
    upload_url = "https://api-app.xtracta.com/v1/documents/upload"
    file = {"userfile": open(filename, mode="rb")}
    data = {
        "api_key": api_key,
        "workflow_id": workflow_id,
        "field_data": classifier_xml,
    }
    r = requests.post(upload_url, data=data, files=file)
    if r.status_code != 200:
        print(r.status_code)
        return t.text
    else:
        response = xmltodict.parse(r.text)
        return response["xml"]["document_id"]
    
test_document_id = upload_file(api_key, header_workflow, ip / test_file)
test_document_id

'99422405'

In [45]:
def get_document(api_key: str, document_id: str):
    
    """retrieves the full xml document from Xtracta and converts it to a dict"""
    
    documents_url = "https://api-app.xtracta.com/v1/documents"
    data = {"api_key": api_key, "document_id": document_id}
    try:
        r = requests.post(documents_url, data=data)
        response = xmltodict.parse(r.text)
        return response
    except Exception as e:
        return e.args
    
test_document = get_document(api_key, test_document_id)

In [46]:
def get_xtracta_status(
    api_key: str,
    workflow_id: str,
    status: str,
    api_download_status: str = "active",
    detailed: int = 0,
    documents_order: str = "asc",
) -> list:
    """Returns a list of all Xtracta documents with a particular status"""
    documents_url = "https://api-app.xtracta.com/v1/documents"
    data = {
        "api_key": api_key,
        "workflow_id": workflow_id,
        "document_status": status,
        "api_download_status": api_download_status,
        "items_per_page": 1000,
        "detailed": detailed,
        "documents_order": documents_order,
    }
    try:
        r = requests.post(documents_url, data=data)
        response = xmltodict.parse(r.text)
    except Exception as e:
        return [e.__str__]

    try:
        response_content = response["documents_response"]["document"]
        if type(response_content) == list:
            return response_content
        else:
            return [response_content]
    except Exception as e:
        if type(e).__name__ == "KeyError":
            return [f"No {status} documents in queue"]
        else:
            return [e]
        
test_reject_list = get_xtracta_status(api_key, header_workflow, 'reject')
test_reject_list

[OrderedDict([('@revision', '2'),
              ('document_id', '99419068'),
              ('document_status', 'reject'),
              ('number_of_pages', '1'),
              ('api_download_status', 'active'),
              ('free_form', None),
              ('classification', 'full'),
              ('classification_class', '1049514'),
              ('classification_design', '0'),
              ('rejection',
               OrderedDict([('reason',
                             [OrderedDict([('message',
                                            'Value is not present in "db_po_number" column of the "POs" database'),
                                           ('validation_rule',
                                            OrderedDict([('type',
                                                          'DATABASE_LIST')])),
                                           ('linked_field',
                                            OrderedDict([('field_id',
                                       

In [ ]:
# def find_documents_to_skip(api_key, header_workflow):
    
#     """You only want to process documents that have data in the document body. 
#     This function finds documents that are not in this state"""
    
#     status_to_skip = ['reject', 'preprocessing', 'output-in-progress']
#     items_to_skip = []
#     for status in status_to_skip:
#         queue = get_xtracta_status(api_key, header_workflow, status)
#         for item in queue:
#             if item != f'No {status} documents in queue':
#                 items_to_skip.append(item['document_id'])
#     return items_to_skip      

## Build the output dictionary from Xtracta data

In [66]:
def create_output(document: Dict[Any, Any]) -> Dict[Any, Any]:
    """Returns a dictionary with document_id, status and version as top level values 
    and remaining fields as key value pairs in a header section"""
    output = {}
    header_dict = document["documents_response"]["document"]["field_data"]["field"]
    header = transform_dict(header_dict)
    output["document_id"] = document["documents_response"]["document"]["document_id"]
    output["status"] = document["documents_response"]["document"]["document_status"]
    output["version"] = document["documents_response"]["document"]["@revision"]
    output["header"] = header
    return output


def transform_dict(start_dict):
    end_dict = {}
    for item in start_dict:
        end_dict[item["field_name"]] = item["field_value"]
    return end_dict

test_output = create_output(test_document)
test_output

{'document_id': '99419093',
 'status': 'reject',
 'version': '2',
 'header': {'supplier_abn': None,
  'supplier': None,
  'supplier_id': None,
  'invoice_number': 'INV-3337',
  'po_number': '12345',
  'line_count': None,
  'account_number': None,
  'invoice_date': '2016-01-25',
  'net_total': '85.00',
  'gst_total': '8.50',
  'gross_total': '93.50',
  'freight': '0.00',
  'bsb_number': '4321432',
  'bank_account_number': '12341234',
  'ok_to_process': '0',
  'period_start_date': None,
  'period_end_date': None,
  'abn_from_db_by_po': None,
  'filename': 'invoice.pdf'}}

In [49]:
def get_documents_wo_json(folder):
    json_files = []
    pdfs = []
    json_list = list(folder.glob("*.json"))
    pdf_list = list(folder.glob("*.pdf"))
    for file in json_list:
        json_files.append(file.stem)
    for pdf in pdf_list:
        pdfs.append(pdf.stem)
    new_documents = list(set(pdfs) - set(json_files))
    return new_documents

get_documents_wo_json(ip)

['invoice']

In [50]:
def open_document_ui(api_key: str, document_id: str) -> str:
    """Opens the Xtracta UI to fix and train documents"""
    documents_url = "https://api-app.xtracta.com/v1/documents/ui"
    data = {
        "api_key": api_key,
        "document_id": int(document_id),
        "buttons": "output,archive",
        "no_lockout": 1,
        "expire": 86400,
    }
    r = requests.post(documents_url, data=data)
    response = xmltodict.parse(r.text)
    return response["documents_response"]["url"]

open_document_ui(api_key, test_document_id)

'https://api-app-ui.xtracta.com/main/an_entry/index/d/UU0OU.AlJ0f91RQjwh2RsaOsI4OMTrxiMR7QWFnAERlVkaEPlCBoOVQTBMJ.zPgoTtSk1ByimMs-'

In [53]:
def update_document(
    api_key: str, document_id: str, delete: int = 0, api_download_status: str = "active"
) -> Dict[str, str]:
    """Updates document on Xtracta"""
    documents_url = "https://api-app.xtracta.com/v1/documents/update"
    data = {
        "api_key": api_key,
        "document_id": int(document_id),
        "delete": delete,
        "api_download_status": api_download_status,
    }
    r = requests.post(documents_url, data=data)
    response = xmltodict.parse(r.text)
    return response["documents_response"]

update_document(api_key, test_document_id, api_download_status='active')

OrderedDict([('status', '200'),
             ('message', 'The request has been successfully processed')])

In [56]:
def get_lines(document):
    lines_dict = document['documents_response']['document']['field_data']['field_set']['row']
    lines = []
    if len(lines_dict) > 1:
        for line_dict in lines_dict:
            line = transform_dict(line_dict['field'])
            lines.append(line)
    else:
        line = transform_dict(lines_dict['field'])
        lines.append(line)
    return lines
    
test_line_document_id = upload_file(api_key, line_workflow, ip / 'invoice.pdf')
test_line_document = get_document(api_key, test_line_document_id)
test_lines = get_lines(test_line_document)
test_lines

[{'po_item': None, 'description': None, 'qty': None, 'total': None}]

In [ ]:
# def build_out_reject_queue(api_key, queue):
#     all_output = []
#     if type(queue) == list:
#         for item in queue:
#             row = {}
#             messages = []
#             document_id = item['document_id']
#             document = get_document(api_key, document_id)
#             output = create_output(document)
#             field_ids = get_field_ids(document)
#             if type(item['rejection']['reason']) != list:
#                 field_id = item['rejection']['reason']['linked_field']['field_id']
#                 message = item['rejection']['reason']['message']
#                 messages.append({'field': field_ids[field_id], 'message': message})
#             else:
#                 for sub_item in item['rejection']['reason']:
#                     field_id = sub_item['linked_field']['field_id']
#                     message = sub_item['message']
#                     messages.append({'field': field_ids[field_id], 'message': message})
#             row['status'] = output['status']
#             try:
#                 row['stem'] = output['header']['filename'].replace('.pdf', '')
#             except:
#                 row['stem'] = 'Click to review'
#             row['messages'] = messages
#             row['review_url'] = open_document_ui(api_key, document_id)
#             row['image_url'] = get_image_urls(document['documents_response']['document']['image_url'])[0]
#             all_output.append(row)
#     return all_output


# def build_out_processing_queue(api_key, queue):
#     all_output = []
#     if type(queue) == list:
#         for item in queue:
#             row = {}
#             messages = []
#             document_id = item['document_id']
#             document = get_document(api_key, document_id)
#             output = create_output(document)
#             row['status'] = output['status']
#             try:
#                 row['stem'] = output['header']['filename'].replace('.pdf', '')
#             except:
#                 row['stem'] = 'Click to review'
#             row['messages'] = messages
#             all_output.append(row)
#     return all_output


# def build_output_queue(api_key, queue):
#     all_output = []
#     if type(queue) == list:
#         for item in queue:
#             row = {}
#             document_id = item['document_id']
#             document = get_document(api_key, document_id)
#             output = create_output(document)
#             row['status'] = output['status']
#             try:
#                 row['stem'] = output['header']['filename'].replace('.pdf', '')
#             except:
#                 row['stem'] = 'Click to review'
#             row['output'] = output
#             row['image_url'] = get_image_urls(document['documents_response']['document']['image_url'])[0]
#             all_output.append(row)
#     return all_output

## Write JSON files

In [64]:
def write_json_simple(filename, output):
    with open(f"{filename}", "w") as f:
        f.write(json.dumps(output, indent=4))
    return filename

test_json_string = 'invoice_simple.json'
test_json_file = write_json_simple(ip / test_json_string, output)
test_json_file

WindowsPath('C:/Users/hudge/Desktop/2019_projects/pipomatic/hudge/xtracta/input/invoice_simple.json')

In [65]:
def write_json(old_name, new_name, dp, output):
    print(old_name, new_name, dp)
    duplicate_name = dp / new_name.name
    if new_name.exists():
        with open(f"{duplicate_name}", "w") as f:
            f.write(json.dumps(output, indent=4))
    else:
        with open(f"{new_name}", "w") as f:
            f.write(json.dumps(output, indent=4))
    if new_name.exists():
        old_name.unlink()
        return new_name
    else:
        return "File not saved!"
    
write_json(test_json_file, op / test_json_file.name, dp, output)

C:\Users\hudge\Desktop\2019_projects\pipomatic\hudge\xtracta\input\invoice_simple.json C:\Users\hudge\Desktop\2019_projects\pipomatic\hudge\xtracta\output\invoice_simple.json C:\Users\hudge\Desktop\2019_projects\pipomatic\hudge\xtracta\data


WindowsPath('C:/Users/hudge/Desktop/2019_projects/pipomatic/hudge/xtracta/output/invoice_simple.json')

## Get list of rejects and reasons

## Build HTML file for handling rejections

In [ ]:
def create_html_section(data, html_template):
    template = Template(html_template)
    html = template.render(data=data)
    return html


reject_queue_html_template = """
    <div class="column is-full">
        <p><strong>Total number of rejects in queue: {{data.reject_count}}</strong></p>
    </div>
    <div class="column is-full">  
        <h2><strong>{{data.output.header.supplier}}</strong></h2>
        <p><strong>Invoice number:</strong> {{data.output.header.invoice_number}}</p>
    </div>
    <div class="column is-two-fifths">
    <section class="section has-text-right">
        <p><strong>Net:</strong> {{"$%.2f"|format(data.output.header.net_total|float)}}</p>
        <p><strong>GST:</strong> {{"$%.2f"|format(data.output.header.gst_total|float)}}</p>
        <p><strong>Total:</strong> {{"$%.2f"|format(data.output.header.gross_total|float)}}</p>
    </section>
    <section class="section">
        <table class="table">
        <thead><tr><th>Field</th><th>Message</th></tr></thead>
        <tbody>
        {% for message in data.messages %}
        <tr>
            <th>{{message.field}}</th>
            <td>{{message.message}}</td>
        </tr>
        {% endfor %}
        </tbody>
        </table>
    </section>
    </div>
    <div class="column is-three-fifths has-text-centered">
        <p><a href="{{data.review_link}}" target="_blank">Review invoice</a></p>
        <p><img src="{{data.invoice_image}}" alt="Invoice Image" width="250"></p>
    </div>
"""


def get_reject_html(api_key, workflow_id, status, html_template):
    queue = get_xtracta_status(api_key, workflow_id, status)
    reject_count = len(queue)
    document_id = queue[0]["document_id"]
    reasons = queue[0]["rejection"]["reason"]
    messages = get_reject_info(api_key, document_id, reasons)
    document = get_document(api_key, document_id)
    image_url = document["documents_response"]["document"]["image_url"][0]
    output = create_output(document)
    review_link = open_document_ui(api_key, document_id)
    data = {
        "output": output,
        "reject_count": reject_count,
        "review_link": review_link,
        "invoice_image": image_url,
        "messages": messages,
    }
    html = create_html_section(data=data, html_template=html_template)
    return html


In [ ]:
def get_reject_info(api_key, document_id, reasons):
    messages = []
    document = get_document(api_key, document_id)
    field_ids = get_field_ids(document)
    if type(reasons) != list:
        field_id = reasons["linked_field"]["field_id"]
        message = reasons["message"]
        messages.append({"field": field_ids[field_id], "message": message})
    else:
        for sub_item in reasons:
            field_id = sub_item["linked_field"]["field_id"]
            message = sub_item["message"]
            messages.append({"field": field_ids[field_id], "message": message})
    return messages


def get_field_ids(document):
    field_ids = {}
    fields = document["documents_response"]["document"]["field_data"]["field"]
    for field in fields:
        field_ids[field["field_id"]] = field["field_name"]
    return field_ids

## Build output once in output status

In [71]:
def build_out_output(filename, document, output):
    output['stem'] = filename.stem
    output['new_filename'] = f"{output['header']['supplier_id']}-{output['header']['invoice_number']}"
    output['header']['emaildate'] = get_email_date(filename)
    output['document_url'] = document['documents_response']['document']['document_url']
    output['image_urls'] = get_image_urls(document['documents_response']['document']['image_url'])
    return output

def get_email_date(filename):
    year = filename.stem[:4]
    month = filename.stem[4:6]
    day = filename.stem[6:8]
    return f"{year}-{month}-{day}"

def get_image_urls(image_urls):
    if type(image_urls) != list:
        image_urls = [image_urls]
    return image_urls

full_test_output = build_out_output(ip / 'invoice.pdf', test_document, test_output)
full_test_output

{'document_id': '99419093',
 'status': 'reject',
 'version': '2',
 'header': {'supplier_abn': None,
  'supplier': None,
  'supplier_id': None,
  'invoice_number': 'INV-3337',
  'po_number': '12345',
  'line_count': None,
  'account_number': None,
  'invoice_date': '2016-01-25',
  'net_total': '85.00',
  'gst_total': '8.50',
  'gross_total': '93.50',
  'freight': '0.00',
  'bsb_number': '4321432',
  'bank_account_number': '12341234',
  'ok_to_process': '0',
  'period_start_date': None,
  'period_end_date': None,
  'abn_from_db_by_po': None,
  'filename': 'invoice.pdf',
  'emaildate': 'invo-ic-e'},
 'stem': 'invoice',
 'document_url': 'https://web1-akl.xtracta.com/akl_northcote_storage2_datasource4/1/0d/79/iq204510992-eqNNNwy4z74T.pdf',
 'image_urls': ['https://web1-akl.xtracta.com/akl_northcote_storage2_datasource4/1/ad/7f/ds99419093-iq204510992-eqNNNwy4z74T6786-1-800.jpg'],
 'new_filename': 'None-INV-3337'}

## Pull company name and location from filename

In [ ]:
def add_company_location(output):
    company_extract = re.compile(r'.*\[<<(.*)>>\].*')
    company_location = company_extract.match(output['header']['email_subject'])[1]
    try:
        output['company'], output['location'] = company_location.split('-')
    except:
        output['company'] = company_location
        output['location'] = 'NA'
    return output

## Moving files in the file system

In [78]:
def move_to_output(op, filename, dp, output):
    new_name = (op / output['new_filename']).with_suffix('.json')
    save_tif(new_name, output['image_urls'])
    write_json(filename, new_name, dp, output)
    print()
    print(f'Document {filename.stem} moved to output')
    return True

def move_to_junk(jp, filename, output):
    new_name = jp / filename
    write_json(filename, new_name, jp, output)
    print()
    print(f'Document {filename.stem} moved to deleted')
    return True

def create_tif_image(image_urls):
    images = []
    for i, url in enumerate(image_urls):
        r = requests.get(url, stream=True)
        if i == 0:
            im = Image.open(r.raw)
        else:
            images.append(Image.open(r.raw))
    return im, images


def save_tif(new_filename, image_urls):
    new_name = op / new_filename.with_suffix('.tif')
    im, images = create_tif_image(image_urls)
    im.save(f'{new_name}', save_all=True, append_images=images)
    return im, images

move_to_output(jp, ip / 'invoice.pdf', full_test_output)
move_files(sp, 'invoice.pdf', ip)
move_to_junk(op, ip / 'invoice.pdf', dp, full_test_output)

NameError: name 'jp' is not defined

## Create TIF files

If the end system requires TIF file storage rather than PDF file storage, these functions create TIF files from the Xtracta image URLs.

In [ ]:
def create_tif_image(image_urls):
    images = []
    for i, url in enumerate(image_urls):
        r = requests.get(url, stream=True)
        if i == 0:
            im = Image.open(r.raw)
        else:
            images.append(Image.open(r.raw))
    return im, images


def save_tif(new_filename, image_urls):
    new_name = op / new_filename.with_suffix('.tif')
    im, images = create_tif_image(image_urls)
    im.save(f'{new_name}', save_all=True, append_images=images)
    return im, images

## Formatting XML for upload into Xtracta's database

Take a list of dicts and format it for uploading to Xtracta's database API

In [ ]:
def update_database_data(api_key, database_id, out, refresh):
    documents_url = 'https://api-app.xtracta.com/v1/databases/data_add'
    data = {'api_key': api_key, 'database_id': int(database_id), 'data': out, 'refresh': refresh}
    r = requests.post(documents_url, data=data)
    response = xmltodict.parse(r.text)
    return response

In [ ]:
def build_xml_data(supplier_data_dict):
    xml_rows = []
    for row in supplier_data_dict:
        po = {'column': [{'@id': '55261', '#text': f"{row['po_number']}"}, 
                         {'@id': '55264', '#text': f"{row['supplier_number']}"},
                         {'@id': '60223', '#text': f"{row['line_number']}"},
                         {'@id': '58133', '#text': f"{row['abn']}"},
                         {'@id': '58134', '#text': f"{row['bsb']}"},
                         {'@id': '58135', '#text': f"{row['bank_account']}"},
                         {'@id': '58242', '#text': f"{row['supplier_name']}"}]}
        xml_rows.append(po)
    xml_data = {'xml': {'row': xml_rows}}
    return xmltodict.unparse(xml_data, pretty=True)

## Build code

The remaining cells load the code to PIP

In [ ]:
!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['build']" \
    --TemplateExporter.exclude_output=True \
    --to python "xtracta.ipynb"

first_line = """'Test flit package'

__version__ = '1.3'

"""
script_file = Path.cwd() / 'xtracta.py'
script = script_file.read_text()
script_file.write_text(first_line + script)
username = script_file.parent.parent.name
system_name = script_file.parent.name
standardised_script_name = f'juggernautomatic_{username}_{system_name}.py'
script_file.replace(script_file.parent / standardised_script_name)
standardised_script_name
# 
#

In [ ]:
!black "juggernautomatic_hudge_xtracta.py"

In [ ]:
# !flit publish # FLIT_PASSWORD="os.getenv('XTRACTA_API_KEY')"